#**Answer Generation 任務**
資料集引用台達閱讀理解資料集









### pytorch環境設定(https://pytorch.org)
選擇適合的版本並複製指令貼到本專案

In [1]:
!pip install torch torchvision

In [2]:
#確定有安裝成功
import torch
print(torch.__version__)
torch.cuda.is_available()

1.5.0+cu101


True

### **取得 google drive 存取權限**

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
cd /content/drive/My Drive/Colab Notebooks/Bert/AnswerGeneration/

/content/drive/My Drive/Colab Notebooks/Bert/AnswerGeneration


In [5]:
!ls

train.ipynb


In [0]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### **下載dataset**

In [7]:
import os
def fileExists(filepath):
  return  os.path.exists(filepath)
#我把資料集放到我的github
if fileExists('train.json')!=True:
  !wget -O train.json https://raw.githubusercontent.com/harry83528/NLP-BERT-Experience/master/AnswerGeneration/DataSet/DRCD_training.json

--2020-05-31 11:58:33--  https://raw.githubusercontent.com/harry83528/NLP-BERT-Experience/master/AnswerGeneration/DataSet/DRCD_training.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15094666 (14M) [text/plain]
Saving to: ‘train.json’

train.json          100%[===================>]  14.39M  63.4MB/s    in 0.2s    

2020-05-31 11:58:34 (63.4 MB/s) - ‘train.json’ saved [15094666/15094666]



In [13]:
!ls

train.ipynb  train.json


In [9]:
!pip install ijson

     |████████████████████████████████| 102kB 2.7MB/s 


In [0]:
import ijson

context = []
question = []
answersText = []
count=0
file_name="train.json"
contextId=''
with open(file_name) as file:
    parser = ijson.parse(file)
    contextId=''
    context2=''
    for prefix, event, value in parser:
        # print('prefix',prefix)
        # print('event',event)
        # print('value',value)
        if prefix=="data.item.paragraphs.item.id":
          #print('contextId',value)          
          contextId=value
        if prefix=="data.item.paragraphs.item.context":
          #print('context',value)
          context2=value
        if prefix=="data.item.paragraphs.item.qas.item.question":
          context.append(context2)
          #print('question',value)
          #print('contextId',contextId)
          question.append(value)
        if prefix=="data.item.paragraphs.item.qas.item.answers.item.text":
          #print('context2',context2)
          #print('answersText',value)
          #print('contextId',contextId)
          answersText.append(value)
        #print("---"*50)
        # count=count+1
        # if count==51:
        #   break;

In [55]:
print(len(context),len(question),len(answersText))
print(context[2000][0:450])
print(question[2000][0:42])
print(answersText[2000][0:16])

26936 26936 26936
賽德娜擁有離心率非常大的橢圓公轉軌道，它的近日點及遠日點估計分別約為76天文單位及937天文單位，是天文學家觀測到的天體中近日點距離太陽最遙遠的一個。賽德娜在天文學家發現它的時候正接近近日點，當時距離太陽為89.6天文單位，是人類觀測到距離太陽最遠的太陽系天體。鬩神星後來在距離太陽97天文單位的位置被天文學家發現，比發現賽德娜的位置更遠。有一些長週期彗星會運行到比賽德娜更遠的位置，不過因為太過黑暗而無法觀測。即使賽德娜在2076年抵達近日點，太陽仍然只是天空中一個非常明亮的恆星，只比滿月還要明亮100倍，而且因為太過遙遠，所以無法用肉眼觀測到圓盤面。賽德娜的公轉周期約為11,400年，會在2075年末至2076年中之間通過近日點，而賽德娜也會在2114年追過鬩神星，成為距離太陽最遠的球狀天體。
被發現時比賽德娜被發現時的距離還遠的星為？
鬩神星


### **安裝所需的函式庫-HuggingFace 團隊將 GitHub 專案大翻新並更名成 transformers(Install the Hugging Face Library)**
https://github.com/huggingface/transformers

In [0]:
!pip install transformers

In [0]:
%tensorflow_version 2.x

### **指定想要用的模型是哪一種**
到以下網址查詢模型與函數資訊
https://huggingface.co/transformers

## **引入需要⽤到的函式庫**

In [0]:
import os
import pickle
import torch
from transformers import BertConfig, BertTokenizer, BertForMaskedLM, AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F # 激勵函數

### **載入資料並將資料轉換成輸入格式(Token Embeddings、Segment Embeddings、Position Embeddings)**

### **輸入格式轉Dataset**

## **確定 Embedding 是否正確**

## **分割數據**
把資料切成訓練集跟測試集
對於 Supervised Learning 來說，我們使⽤標註資料（Labeled Data）來做訓練
因此我們需要訓練資料（Training Data）
但是我們蒐集到的資料卻不能全部拿來做訓練
因為我們必須要保留⼀些當作測試資料（Testing Data）來評估模型表現
這些資料必須跟訓練資料是完全不同的，否則就有作弊的嫌疑
所以我們會把我們的資料切成訓練集與測試集，通常會保留比較多當作訓練資料
（60%~80%），其餘做測試資料

In [0]:
def split_dataset(full_dataset, split_rate=0.8):  
    train_size = int(split_rate * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
    return train_dataset,test_dataset

### **Fine-Tuning**


In [0]:
# 計算正確值
def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

### **開始訓練**

### **儲存模型**